In [1]:
class Logistic_Regression():
    def __init__(self):
        #initializing the class (like import numpy as np)
        #How to code in Jupyter: LR = Logistic_Regression()
        print("LR class activated")
        
    
    
    def standardize_values(self, mat, column_index_list):
        #standardization method
        #input the data and then a list containing the column indexes to be standardized
        for i in range(len(column_index_list)):
            mat[:,column_index_list[i]] = (mat[:,column_index_list[i]] - mat[:,column_index_list[i]].mean())/mat[:,column_index_list[i]].std()
        return mat
    
        
        
    def OHE_from_data(self, data, column_index_list):
        #One Hot Encoding directly from data
        """
        Column Indexes in list MUST be in descending order
        """
        mat = np.asmatrix(pd.read_csv(data)) #reads data as matrix
        for i in range(len(column_index_list)):
            num_of_categories = max(mat[:,column_index_list[i]]) #Finding number of OHE columns to add
            N,D = mat.shape
            zeros = np.zeros(N*int(num_of_categories)).reshape(N,int(num_of_categories)) #Making OHE columns of 0
            ohe_mat = np.hstack((mat,zeros)) #Concatenating the data
        
            for num in range(N):
                t = int(ohe_mat[num,column_index_list[i]])
                ohe_mat[num, (D - 1) + t] = 1 
            ohe_mat = np.delete(ohe_mat, column_index_list[i], axis = 1) #dropping the original categorical column
        
        return ohe_mat.astype(float)
        
    
    def OHE_from_mat(self, mat, column_index_list):
        #One Hot Encoding if a matrix is used instead
        """
        Column Indexes in list MUST be in descending order
        """
        for i in range(len(column_index_list)):
            num_of_categories = max(mat[:,column_index_list[i]]) #Finding number of OHE columns to add
            N,D = mat.shape
            zeros = np.zeros(N*int(num_of_categories)).reshape(N,int(num_of_categories)) #Making OHE columns of 0
            ohe_mat = np.hstack((mat,zeros)) #Concatenating the data
        
            for num in range(N):
                t = int(ohe_mat[num,column_index_list[i]])
                ohe_mat[num, (D - 1) + t] = 1 
            ohe_mat = np.delete(ohe_mat, column_index_list[i], axis = 1) #dropping the original categorical column

        return ohe_mat.astype(float)
    
    def auto_z_make(self, mat):
        #Method to make Z; the method creates the weights for you
        N,D = mat.shape
        W = np.random.randn(D)
        return np.dot(W,mat.T)
    
    def z_make(self, mat,W):
        #making Z using your own weights
        return np.dot(W,mat.T)
    
    def sigmoid(self, z):
        #predictions aka sigmoid curve
        return 1/(1 + np.exp(-z))
    
    def cost_function(self, T, Q):
        #T = Targets; Q = Sigma = Predictions
        return np.dot(-(T, np.log(Q).T) + np.dot((1-T),np.log(1-Q).T))
    
    def auto_optimize_weights(self, X,Y,iter_num,a,lamb):
        #Gradient Descent and Cost Regularlization. Computer comes up with Weights
        N,D = X.shape
        W = np.random.randn(D) #Making the weights
        predictions = self.sigmoid(np.dot(W,X.T))
        print(np.mean(np.round(predictions) == Y)) #initial accuracy of model (might remove later)
    
        for i in range(iter_num):
            predictions = self.sigmoid(np.dot(W,X.T))
            derivative = np.dot((predictions - Y), X) + lamb*W
            W -= a*derivative.astype(float) #Gradient Descent
        
        print(np.mean(np.round(predictions) == Y)) 
        return W
    
    def optimize_weights(self, X,Y,iter_num,a,lamb,W):
        #Gradient Descent using your own Weights
        predictions = self.sigmoid(np.dot(W,X.T))
        print(np.mean(np.round(predictions) == Y)) #initial accuracy
    
        for i in range(iter_num):
            predictions = self.sigmoid(np.dot(W,X.T))
            derivative = np.dot((predictions - Y), X) + lamb*W
            W -= (a*derivative).astype(float) #Gradient Descent
        
        print(np.mean(np.round(predictions) == Y))
        return W
    
    def accuracy_check(self, predictions, targets):
        return np.mean(np.round(predictions) == targets)

    def remove_NaN_Numerical(self, mat,column_index_list):
        N,D = mat.shape
        for num in range(len(column_index_list)):
            temporary_mat = mat[:,column_index_list[num]].astype(float)
            mean = temporary_mat[~np.isnan(temporary_mat)].mean()
            for i in range(N):
                if np.isnan(mat[i,column_index_list[num]]):
                    mat[i,column_index_list[num]] = mean
        return mat

    def remove_NaN_Categorical(self, mat,column_index_list):
        N,D = mat.shape
        for num in range(len(column_index_list)):
            temporary_mat = mat[:,column_index_list[num]].astype(float)
            mean = temporary_mat[~np.isnan(temporary_mat)].mean()
            for i in range(N):
                if np.isnan(mat[i,column_index_list[num]]):
                    mat[i,column_index_list[num]] = np.round(mean)
        return mat
    
    def remove_NaN_Pandas(self, df):
        """
        Remove NaN values while still in DataFrame (df must be a dataframe).
        Does not work for categorical columns
        """
        df.fillna(df.mean(), inplace = True)
        #fillna = fill NaN. inplace = True to permanantly change the data
    
    
    def make_model(self, X_train, X_test, y_train, y_test, nameOfTargets, M = 100, iteration_num = 1000, a = 0.02,):
        """
        default: num nodes = 100, num iterations = 1000, learning rate = 0.02
        for nameOfTarets, use df.Species.unique() to get all unique categories. Drop as necessary.
        """
        np.random.seed(1) #making sure the weights are the same every time the cell is rerun (still random)
        N,D = X_train.shape #N = num subjects, D = num features
        M = 100 #num hidden nodes of the hidden layer
        K = nameOfTargets.shape[0] #number of outputs

        #creating the weights (randomly)
        W = np.random.randn(D*M).reshape(D,M)
        V = np.random.randn(M*K).reshape(M,K)


        #creating the biased terms
        b = np.random.randn(M).reshape(1,M) #Generating biased terms for hidden nodes
        ones = np.ones(N).reshape(N,1) #Making an Nx1 matrix to multiply with the biased terms.
        b = np.dot(b_ones,b) #After multiplying, NxM matrix (each row is a subject, each column is a node). The biased terms of each column are the same (biased term of each node does not change for the subject).

        c = np.random.randn(K).reshape(1,K) #Repeating the process for the biased terms of the output layer. NxK matrix once completed.
        c = np.dot(c_ones, c)

        for j in range(iteration_num): #Back Propagation

            #feed forward
            z = np.dot(X_train,W) + b
            z = 1/(1 + np.exp(-z.astype(float)))
            predictions = np.exp(np.dot(z,V) + c)

            #softmax
            for i in range(predictions.shape[0]):
                predictions[i,:] = predictions[i,:]/np.sum(predictions[i,:])


            #gradient descent (all from formula)

                #Calculating all the partial derivatives of cost
            dV = np.dot(z.T,(y_train - predictions)) 
            dZ = np.dot(np.dot(np.dot((y_train - predictions), V.T).T, z),(1-z.T)) #will be used to calculate dW
            dW = np.dot(X_train.T,dZ.T) 
            db = np.dot(np.dot(np.dot((y_train - predictions), V.T).T, z), (1-z.T)).T.sum(axis = 0) 
            dc = (y_train - predictions).sum(axis = 0) 


            W += a*dW.astype(float) #Gradient Descent
            V += a*dV.astype(float) 
            b += a*db.astype(float)
            c += a*dc.astype(float)

            if j%100 == 0: #Every 100 iterations, print out the cost and accuracy
                total = -np.dot(y_train.T, np.log(predictions))
                cost = total.sum() #Cost of the model
                Accuracy = np.mean(np.round(predictions) == y_train) #Accuracy of the model
                print(cost, Accuracy)


        print(" ")
        print("Final Cost and Accuracy of training data: ")
        print(cost, Accuracy)

        #Applying the model to the test data. The X_test data must be put through the softmax function and compared to y_test

        #feed forward
        z = np.dot(X_test,W) + b[0] #using only b causes a dimension error. Using b[0] gets all the relevant biased terms and makes use of how numpy array addition works.
        z = 1/(1 + np.exp(-z.astype(float)))
        test_predictions = np.exp(np.dot(z,V) + c[0]) #same thing with c[0]

        #softmax
        for i in range(test_predictions.shape[0]):
            test_predictions[i,:] = test_predictions[i,:]/np.sum(test_predictions[i,:])

        test_Acc = np.mean(np.round(test_predictions) == y_test)
        test_total = -np.dot(y_test.T, np.log(test_predictions))
        test_cost = total.sum()

        print(" ")
        print("Cost and Accuracy of testing data: ")
        print(test_cost, test_Acc)
        
        self.b = b
        self.c = c
        self.W = W
        self.V = V
        self.nameOfTargets = nameOfTargets
    def classify(self, data, X_new):
    """
    X_new is an unstandardized copy of the original data
    """
    for i in range(X_new.shape[1]):
        data[:,i] = (data[:,i].astype(float) - np.mean(X_new[:,i].astype(float)))/np.std(X_new[:,i].astype(float)) 
        #Standardizing inputs using X_new, which was created earlier in the algorithm
    
    #Passing the inputs through feed forward
    z = np.dot(data,self.W) + self.b[0]
    z = 1/(1 + np.exp(-z.astype(float)))
    test_predictions = np.exp(np.dot(z,self.V) + self.c[0])

    #softmax
    for i in range(test_predictions.shape[0]):
        test_predictions[i,:] = test_predictions[i,:]/np.sum(test_predictions[i,:])
    
    test_predictions = np.round(test_predictions) #rounding the predictions to only get 1 and 0
    j = np.where(test_predictions == 1)[1][0] #Getting the index of which target we classified the data as (which target node has a value of 1)
    
    
    return nameOfTargets[j]#Returning the name of the target
